<a href="https://colab.research.google.com/github/suyeon-9706/MNIST/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorch 
- Scientific Computing Package based on Python (Deep Learning Library)
- Dynamic Computing Graph (DCG) Support
- Easy access to python arrangement features
- Excellent compatibility with numpy / scipy.
- Most are implemented in python except C++ code for tensor operation.

# MNIST
- Provided on Yann LeCun's website
- A simple computer vision data set
- It consists of handwritten images.
- Each data is labeled 'What is the number of that data' with the data.






In [0]:
#@title Install pytorch
!pip install torch
!pip install torchvision

In [0]:
import torch
import torch.nn as nn # Packages for creating a Neural Network
# torch.autograd: A package that is central to the Neural network, providing automatic differentiation for all operations of the Tensor

from torch.autograd import Variable # Variable class: core class of autograd package

# torchvision: use for image classification training, easy to vision training
import torchvision.datasets as dsets # Data loader for datasets such as CIFAR10, MNIST, etc.
import torchvision.transforms as transf # A package that transforms images of PIL type into torch tensor type

In [0]:
import matplotlib.pyplot as plt # visualization package(Used to draw a graph)
import numpy as np # For matrix operations
import random #For random operations

In [0]:
#@title Loading MNIST data
mnist_train = dsets.MNIST(root='data/', train=True, transform=transf.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='data/', train=False, transform=transf.ToTensor(), download=True)